In [47]:
!pip install googletrans==4.0.0-rc1
!pip install pandas 
!pip install spacy 
!pip install unicodedata 
!pip install nltk
!pip install gensim
!python3 -m spacy download en_core_web_sm

ERROR: Could not find a version that satisfies the requirement unicodedata (from versions: none)
ERROR: No matching distribution found for unicodedata
2023-03-29 15:19:48.460930: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-29 15:19:48.541752: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-29 15:19:48.541769: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-29 15:19:48.920282: W tensorflow/compiler/xla/stream_executor/platform/default/dso_lo

In [48]:
import pandas as pd
from googletrans import Translator
from tqdm import tqdm
import spacy
import unicodedata
import re
import nltk
from gensim.parsing.preprocessing import strip_punctuation

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/vincenzo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [49]:
flag = False

In [50]:
def get_new_df(df):
    translator = Translator()
    D = {
        'Job_offer':[],
        'Label':[],
        'Traslation':[]
    }
    for _, row in tqdm(df.iterrows(), total=df.shape[0]):
        try:
            traslation = translator.translate(str(row['Job_offer']), src='it', dest='en').text
        except:
            continue
        D['Job_offer'].append(row['Job_offer'])
        D['Label'].append(row['Label'])
        D['Traslation'].append(traslation)
    new_df = pd.DataFrame.from_dict(D)
    return new_df

In [51]:
sp = spacy.load('en_core_web_sm',  disable=['parser', 'ner'])

def add_token_col(df):
    D = {
        'Job_offer':[],
        'Label':[],
        'Traslation':[],
        'token':[]
    }
    for _, row in tqdm(df.iterrows(), total=df.shape[0]):
        tokens = None
        message = row['Traslation']
        # Get lemma
        tokens = [token.lemma_ for token in sp(message)]

        # Normalize Unicode String and convert to lowercase
        tokens = [unicodedata.normalize('NFKD', token).lower() for token in tokens]

        #print('Removing all but chars and numbers...')
        tokens = [re.sub(r'[\W_]+', '',token) for token in tokens] 

        # Remove numbers, but not words that contain numbers.
        tokens = [token for token in tokens if not token.isnumeric()]

        # Remove words that are only one or two characters.
        tokens = [token for token in tokens if len(token) > 2]

        # Remove stopwords 
        stop_words = stopwords.words('english')
        tokens = [word for word in tokens if word not in stop_words]

        # Strip punctuation
        tokens = [strip_punctuation(token) for token in tokens]
        D['Job_offer'].append(row['Job_offer'])
        D['Label'].append(row['Label'])
        D['Traslation'].append(row['Traslation'])
        D['token'].append(' '.join(tokens))
    new_df = pd.DataFrame.from_dict(D)
    return new_df

In [52]:
if flag:
    train_df = pd.read_csv('train_set.csv')
    test_df = pd.read_csv('test_set.csv')
    new_train_df = get_new_df(train_df)
    new_train_df.to_csv('new_train_set.csv', index=False)
    new_test_df = get_new_df(test_df)
    new_test_df.to_csv('new_test_set.csv', index=False)
    new_train_df = add_token_col(new_train_df)
    new_train_df.to_csv('new_train_set.csv', index=False)
    new_test_df = add_token_col(new_test_df)
    new_test_df.to_csv('new_test_set.csv', index=False)
else:
    new_train_df = pd.read_csv('new_train_set.csv')
    new_test_df = pd.read_csv('new_test_set.csv')